In [1]:
import sqlite3 as sql
import math
import catalog_utils as cu

In [2]:
def init(con, source):
    cur = con.cursor()
    target = source+'_pa'
    cur.execute('''CREATE TABLE '''+ target +''' AS 
                   SELECT desig, m3_6, m4_5, m5_8, m8_0, 
                   c3645, c3680, 
                   Phase, Type, f_Type
                   FROM '''+ source +''' WHERE Type = 1 OR Type = 2''')
    return target

In [3]:
def extract_12(con, source):
    cur = con.cursor()
    target = source+'_12'
    cur.execute('''CREATE TABLE '''+ target +''' AS SELECT desig FROM '''+ source +''' WHERE
                   (m3_6 > 3 AND 
                    m3_6 < 9.5 AND 
                    c3645 > 0.2 AND 
                    c3645 < 1.25) 
                   OR
                   (m3_6 > 3.5 AND 
                    m3_6 < 9.5 AND 
                    c3680 > 0.4 AND 
                    c3680 < 2.6)''')
    return target

In [4]:
def extract_uc(con, source, cutoff36):
    cur = con.cursor()
    target = source + '_uc'
    cur.execute('''CREATE TABLE '''+ target + ''' AS SELECT desig FROM '''+ source + ''' WHERE 
                   Phase = 2 AND 
                   (m5_8 IS NULL OR m8_0 IS NULL) AND
                   m3_6 > ?''', [cutoff36])
    return target

In [5]:
def run_phaseagb(master, constr, cutoff36):
    con = sql.connect(constr)
    
    wc = init(con, master)
    cu.update_phase(con, master, wc, 3)
    
    c12 = extract_12(con, wc)
    cu.update_type(con, master, c12, 12)
    cu.update_type(con, wc, c12, 12)
    
    cuc = extract_uc(con, wc, cutoff36)
    cu.update_flag(con, master, cuc, 'uc')
    cu.update_flag(con, wc, cuc, 'uc')
    
    return 0

In [6]:
run_phaseagb('gtw49', '/home/kecai/w49/w49test.db', 15.2)

0